<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-secret-whiteboard/blob/master/main_secret_whiteboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>